In [ ]:
#fetching data through the coin market cap website using the API
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'10',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '4f98d702-e9e2-4ea0-9e85-74fce5f04c23',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

#jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10

In [ ]:
type(data)

In [ ]:
#coverting the json format into dataframe
import pandas as pd
pd.set_option("display.max_columns",None)
coin = pd.json_normalize(data["data"])
type(coin)

In [ ]:
#adding new column called time stamp with values of date and time when the data was extracted using api
coin['Time_stamp'] = pd.to_datetime('now')
coin

In [ ]:
#Automating the Data pull
def auto_pull():
    from requests import Request, Session
    from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
    import json
    
    global coin
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'100',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '4f98d702-e9e2-4ea0-9e85-74fce5f04c23',
    }

    session = Session()
    session.headers.update(headers)
    
    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    #converting the json format into dataframe
    import pandas as pd
    myth = pd.json_normalize(data['data'])
    
    #adding a new column of time stamp to the converted dataframe
    myth['Time_stamp'] = pd.to_datetime('now')
    coin = coin.append(myth)
    
    #saving the dataframe into the csv file
    #if file doesn't exist then will create a new file else will append the data to the existing file
    if not os.path.isfile(r'C:\Users\Ritesh\Documents\Crypto API pull\API.csv'):
        coin.to_csv(r'C:\Users\Ritesh\Documents\Crypto API pull\API.csv', header='column_names')
    else:
        coin.to_csv(r'C:\Users\Ritesh\Documents\Crypto API pull\API.csv',mode='a',header=False)
    


In [ ]:
import os
from time import time
from time import sleep

#Iterating the function so as to auto pull the data
for n in range(10):
    auto_pull()
    print("pull {} completed".format(n+1))
    #after each pull the program will go to sleep for one minute
    sleep(60)


In [ ]:
#changing the output display to show the whole data
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
coin

In [ ]:
#reading the saved csv file
df = pd.read_csv(r'C:\Users\Ritesh\Documents\Crypto API pull\API.csv')
df

In [ ]:
#changing the number of digits after decimal or float format according to need
pd.set_option('display.float_format', lambda a: ('%.3f') % a)
df

In [ ]:
#Analysis
df1 = df.groupby('name', sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()
type(df1)

In [ ]:
df1

In [ ]:
df2 = df1.stack()
type(df2)

In [ ]:
df2

In [ ]:
df3 = df2.to_frame('percentage_change')
type(df3)

In [ ]:
df3

In [ ]:
df4 = df3.reset_index()
df4

In [ ]:
df5=df4.rename(columns={'level_1':'time'},inplace=False)
df5

In [ ]:
df5['time'] = df5['time'].replace(['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d'],['1hr','24hr','7d','30d','60d','90d'])
df5

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#trend of various crypto currency over last 90 days
sns.catplot(y="percentage_change", x="time",  hue='name', kind='point', data=df5 )

In [ ]:
df6 = df[['name','symbol','quote.USD.price','quote.USD.volume_24h','quote.USD.volume_change_24h','Time_stamp']]
df6

In [ ]:
#volume change distribution of last 24 hrs
p1 = sns.catplot(x="symbol", y="quote.USD.volume_change_24h",hue='name', kind='point', data=df6)
p1.set_axis_labels('coin','percent volume change')
p1

In [ ]:
df7 = df6.query("name == 'Ethereum'")
df7

In [ ]:
#ethereum price change
sns.set_theme(style='darkgrid')
p2 = sns.lineplot(x="Time_stamp", y="quote.USD.price",data=df7)